In [ ]:
import os
import base64
import langchain
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [ ]:
with open("../data/secrets.json") as secrets:
    secrets_dict = eval(secrets.read())
    open_api_key = base64.b64decode(secrets_dict["openai_api_key"]).decode('ascii')
    openai_organization = base64.b64decode(secrets_dict["organization_id"]).decode('ascii')
os.environ["OPENAI_API_KEY"] = open_api_key
os.environ["OPENAI_ORGANIZATION"] = openai_organization

from chatrag.retriever import create_retriever_from_csv, create_retrieval_chain

In [ ]:
langchain.debug = True

## Data preparation

In [ ]:
# OpenAI saved encoded in base64 in a single-line file at data folder
with open("../data/secrets.txt") as secrets:
    open_ai_key = secrets.readline()
    open_ai_key = base64.b64decode(open_ai_key).decode('ascii')
os.environ["OPENAI_API_KEY"] = open_ai_key

In [ ]:
retriever_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=2000)
# retriever_llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=2000)
retriever = create_retriever_from_csv(
    csv_path="../data/movies_title_overview_vote.csv",
    metadata_columns_dtypes={"vote_average": "float"},
    llm=retriever_llm
)

# Vectorstore DB

In [ ]:
retrieval_chain_llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=1000)
# retrieval_chain_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=2000)

In [ ]:
g_media_sq_retrieval_chain = create_retrieval_chain(retriever=retriever, llm=retrieval_chain_llm)

In [ ]:
g_media_sq_retrieval_chain.run("I want to watch a movie about space battles.")

# Tooling

In [ ]:
calc_tool = load_tools(["llm-math"], llm=OpenAI(temperature=0))

In [ ]:
retriever_description = """Movie search tool. The action input must be just topics in a natural language sentence"""

In [ ]:
tools = [
    Tool(
        name = "Search movies",
        func=g_media_sq_retrieval_chain.run,
        description=retriever_description
    ),
    calc_tool[0]
]

# Prompt templates

In [ ]:
prefix = """ You are an assistant expert in movies recommendation. You should guide and help the user through the whole process until suggesting the best movie options to watch. You should attend to all the user requirements always taking into account user data. 

For the first interactions you should collect some user configuration data. This data will restrict the movies to consider.

User Data to collect (mandatory):
    Target genre: Movie genre e.g. fiction, adventure, trhiller...
    Movie overview topic: List of keywords defining the campaign context.

After succesfully collecting data, you should keep the conversation with the human, answering the questions and requests as good as you can. To do so, you have access to the following tools:"""

In [ ]:
suffix = """Begin! Your first action must ve collect user data and keep it. Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB``` then Observation: ... Thought: ... Action: ..."""


In [ ]:
chat_history = MessagesPlaceholder(variable_name="chat_history")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_kwargs = {
        "memory_prompts": [chat_history],
        "input_variables": ["input", "agent_scratchpad", "chat_history"],
        "prefix": prefix,
        "suffix": suffix
    }

# Chains

In [ ]:
# agent_llm_model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=1000)
agent_llm_model = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=2000)

In [ ]:
general_chat_agent = initialize_agent(
    tools=tools,
    llm=agent_llm_model,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    agent_kwargs=agent_kwargs,
)

# TESTS

### Run

In [ ]:
langchain.debug = True

In [ ]:
memory.chat_memory.messages

In [ ]:
response = general_chat_agent.run(input="Hi, my name is Aitor")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="""I want to watch a movie about outer space exploration.""")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
response = general_chat_agent.run(input="A thriller set in space sounds good.")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
print(response)

In [ ]:
# TODO: Current issue. Though it identifies what to do, doesn't launch the action tool. 
response = general_chat_agent.run(input="space exploration")
print("\nA partir de aquí la respuesta:\n")
print(response)

In [ ]:
print(response)

In [ ]:
response = general_chat_agent.run(input="search it")
print("\nA partir de aquí la respuesta:\n")
print(response)